In [ ]:
import os
import sys
import fnmatch
import numpy as np
import pandas as pd
import json
import gzip
import pickle
import csv
import scipy.sparse
Xauth = None
from collections import defaultdict

In [ ]:
papers = pickle.load(gzip.open('parsed_files.pkl.gz','rb'))

In [ ]:
tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = papers[0]
print(papers[0],len(papers))

In [ ]:
informal_venues = {}
total_venues = {}
pubtypes = {}
pagecounts = {}

for paper in papers:
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    total_venues[venue] = 1 + total_venues.get(venue,0)
    pagecounts[pages] = 1 + pagecounts.get(pages,0)
    pubtypes[publtype] = 1
    if publtype is not None:
        informal_venues[venue] = 1 + informal_venues.get(venue,0)

frac_informal_venues = {k:v/total_venues[k] for k,v in informal_venues.items()}
skip_venues = {k:v for k,v in frac_informal_venues.items() if v > 0.5 }

In [ ]:
#sorted([(v,k,frac_informal_venues[k]) for k,v in informal_venues.items()],reverse=True)

In [ ]:
#pubtypes

In [ ]:
#skip_venues

In [ ]:
faculty_affil = pd.read_csv('faculty-affiliations.csv')
ranks = pd.read_csv('other_ranks/ranks.csv')
def csv2dict_str_str(fname):
    with open(fname, mode='r') as infile:
        rdr = csv.reader(infile)
        d = {rows[0].strip(): rows[1].strip() for rows in rdr}
    return d
aliasdict = csv2dict_str_str('dblp-aliases.csv')

In [ ]:
eb_toofew

In [ ]:
useful_papers = []
all_venues = set()
all_authors = set()
for paper in papers:
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    #if 'Jonathan T. Moon' in authors:
    #    print(paper)
    if publtype is None and venue not in skip_venues and (pages != 0 or (not eb_toofew)) and tag != 'incollection' and not eb_skip \
                        and year >= 1970 and year < 2020 and (pages==-1 or pages >=6 or (not eb_toofew)) and (pages <= 100 or (not eb_toofew)):
        authors = [aliasdict.get(a,a) for a in authors]
        #if 'Jonathan T. Moon' in authors:
        #    print('PASSED',paper)
        for a in authors:
            all_authors.add(a)
        all_venues.add(venue)
        useful_papers.append((tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip))

In [ ]:
len(all_venues),len(all_authors),len(useful_papers)

In [ ]:
all_venues = sorted(list(all_venues))
all_authors = sorted(list(all_authors))
useful_papers = sorted(useful_papers,key = lambda paper: (paper[6],paper[3])) #sort by year,conf

In [ ]:
conf_idx = {v:i for i,v in enumerate(all_venues)}
name_idx = {v:i for i,v in enumerate(all_authors)}

In [ ]:
with gzip.open('useful_venue_list.pkl.gz','wb') as fp:
    pickle.dump(all_venues,fp,-1)
with gzip.open('useful_authors_list.pkl.gz','wb') as fp:
    pickle.dump(all_authors,fp,-1)
with gzip.open('useful_papers.pkl.gz','wb') as fp:
    pickle.dump(useful_papers,fp,-1)